In [1]:
#Importing the libraries

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, manhattan_distances, euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from gensim import models
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style
%matplotlib inline
from gensim.models import FastText as ft
from IPython.display import Image

In [2]:
df = pd.read_excel("data.xlsx")
# Viewing Top 5 Rows
df.head(5)

,StockCode,Product Name,Description,Category,Brand,Unit Price
0,22629,Ganma Superheroes Ordinary Life Case For Samsu...,"New unique design, great gift.High quality pla...",Cell Phones|Cellphone Accessories|Cases & Prot...,Ganma,13.99
1,21238,Eye Buy Express Prescription Glasses Mens Wome...,Rounded rectangular cat-eye reading glasses. T...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,19.22
2,22181,MightySkins Skin Decal Wrap Compatible with Ni...,Each Nintendo 2DS kit is printed with super-hi...,Video Games|Video Game Accessories|Accessories...,Mightyskins,14.99
3,84879,Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...,The sheerest compression stocking in its class...,Health|Medicine Cabinet|Braces & Supports,Medi,62.38
4,84836,Stupell Industries Chevron Initial Wall D cor,Features: -Made in the USA. -Sawtooth hanger o...,Home Improvement|Paint|Wall Decals|All Wall De...,Stupell Industries,35.99


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3958 entries, 0 to 3957
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   StockCode     3958 non-null   object 
 1   Product Name  3958 non-null   object 
 2   Description   3958 non-null   object 
 3   Category      3856 non-null   object 
 4   Brand         3818 non-null   object 
 5   Unit Price    3943 non-null   float64
dtypes: float64(1), object(5)
memory usage: 185.7+ KB


In [4]:
df.isnull().sum()

StockCode         0
Product Name      0
Description       0
Category        102
Brand           140
Unit Price       15
dtype: int64

## Download word embeddings

download w2v
gdown https://drive.google.com/file/d/1LDdF6M6VzNhXfJyqb1OXLWno8TrExg6o/view?usp=sharing

download glove
wget https://drive.google.com/file/d/1P9JFZ43Id9dXOo_xFpqLkx81jOsXrQC0/view?usp=sharing

download fastext
fasttext https://drive.google.com/file/d/1hgb-kxJzFkcktsxBEpthmCxzlwe_IM6o/view?usp=sharing

In [5]:
# Importing Word2Vec
word2vecModel = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [6]:
# # Import Glove
glove_df = pd.read_csv('glove.6B/glove.6B.300d.txt', sep=" ",
                       quoting=3, header=None, index_col=0)
glove_model = {key: value.values for key, value in glove_df.T.items()}

In [7]:
count_vectorizer=CountVectorizer(stop_words='english')
tfidf=TfidfVectorizer(stop_words='english',analyzer='word',ngram_range=(1,3))

In [8]:
# Combining Product and Description
df['Description'] = df['Product Name'] + ' ' +df['Description']

In [9]:
# Dropping Duplicates and keeping first record
unique_df = df.drop_duplicates(subset=['Description'], keep='first')

In [10]:
unique_df.head()

,StockCode,Product Name,Description,Category,Brand,Unit Price
0,22629,Ganma Superheroes Ordinary Life Case For Samsu...,Ganma Superheroes Ordinary Life Case For Samsu...,Cell Phones|Cellphone Accessories|Cases & Prot...,Ganma,13.99
1,21238,Eye Buy Express Prescription Glasses Mens Wome...,Eye Buy Express Prescription Glasses Mens Wome...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,19.22
2,22181,MightySkins Skin Decal Wrap Compatible with Ni...,MightySkins Skin Decal Wrap Compatible with Ni...,Video Games|Video Game Accessories|Accessories...,Mightyskins,14.99
3,84879,Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...,Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...,Health|Medicine Cabinet|Braces & Supports,Medi,62.38
4,84836,Stupell Industries Chevron Initial Wall D cor,Stupell Industries Chevron Initial Wall D cor ...,Home Improvement|Paint|Wall Decals|All Wall De...,Stupell Industries,35.99


In [11]:
unique_df['desc_lower']=unique_df['Description'].apply(lambda x: x.lower())

C:\Users\uzmam\AppData\Local\Temp/ipykernel_688/1090978541.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_df['desc_lower']=unique_df['Description'].apply(lambda x: x.lower())


In [12]:
unique_df.head()

,StockCode,Product Name,Description,Category,Brand,Unit Price,desc_lower
0,22629,Ganma Superheroes Ordinary Life Case For Samsu...,Ganma Superheroes Ordinary Life Case For Samsu...,Cell Phones|Cellphone Accessories|Cases & Prot...,Ganma,13.99,ganma superheroes ordinary life case for samsu...
1,21238,Eye Buy Express Prescription Glasses Mens Wome...,Eye Buy Express Prescription Glasses Mens Wome...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,19.22,eye buy express prescription glasses mens wome...
2,22181,MightySkins Skin Decal Wrap Compatible with Ni...,MightySkins Skin Decal Wrap Compatible with Ni...,Video Games|Video Game Accessories|Accessories...,Mightyskins,14.99,mightyskins skin decal wrap compatible with ni...
3,84879,Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...,Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...,Health|Medicine Cabinet|Braces & Supports,Medi,62.38,mediven sheer and soft 15-20 mmhg thigh w/ lac...
4,84836,Stupell Industries Chevron Initial Wall D cor,Stupell Industries Chevron Initial Wall D cor ...,Home Improvement|Paint|Wall Decals|All Wall De...,Stupell Industries,35.99,stupell industries chevron initial wall d cor ...


In [13]:
unique_df['desc_lower']=unique_df['desc_lower'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

C:\Users\uzmam\AppData\Local\Temp/ipykernel_688/3469033122.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_df['desc_lower']=unique_df['desc_lower'].apply(lambda x: re.sub(r'[^\w\s]', '', x))


In [14]:
# Coverting Description to List
desc_list = list(unique_df['desc_lower'])

In [15]:
unique_df= unique_df.reset_index(drop=True)

In [16]:
unique_df.reset_index(inplace=True)

In [17]:
unique_df.head()

,index,StockCode,Product Name,Description,Category,Brand,Unit Price,desc_lower
0,0,22629,Ganma Superheroes Ordinary Life Case For Samsu...,Ganma Superheroes Ordinary Life Case For Samsu...,Cell Phones|Cellphone Accessories|Cases & Prot...,Ganma,13.99,ganma superheroes ordinary life case for samsu...
1,1,21238,Eye Buy Express Prescription Glasses Mens Wome...,Eye Buy Express Prescription Glasses Mens Wome...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,19.22,eye buy express prescription glasses mens wome...
2,2,22181,MightySkins Skin Decal Wrap Compatible with Ni...,MightySkins Skin Decal Wrap Compatible with Ni...,Video Games|Video Game Accessories|Accessories...,Mightyskins,14.99,mightyskins skin decal wrap compatible with ni...
3,3,84879,Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...,Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...,Health|Medicine Cabinet|Braces & Supports,Medi,62.38,mediven sheer and soft 1520 mmhg thigh w lace ...
4,4,84836,Stupell Industries Chevron Initial Wall D cor,Stupell Industries Chevron Initial Wall D cor ...,Home Improvement|Paint|Wall Decals|All Wall De...,Stupell Industries,35.99,stupell industries chevron initial wall d cor ...


In [18]:
def find_euclidean_distances(sim_matrix, index, n=10): 

    # Getting Score and Index
    result = list(enumerate(sim_matrix[index]))

    # Sorting the Score and taking top 10 products
    sorted_result = sorted(result,key=lambda x:x[1],reverse=False)[1:10+1]

    # Mapping index with data
    similar_products =  [{'value': unique_df.iloc[x[0]]['Product Name'], 'score' : round(x[1], 2)} for x in sorted_result]
    
    return similar_products

In [19]:
def find_similarity(cosine_sim_matrix, index, n=10):
    
    # calculate cosine similarity between each vectors
    result = list(enumerate(cosine_sim_matrix[index]))
    
    # Sorting the Score
    sorted_result = sorted(result,key=lambda x:x[1],reverse=True)[1:n+1]
    
    similar_products =  [{'value': unique_df.iloc[x[0]]['Product Name'], 'score' : round(x[1], 2)} for x in sorted_result]
     
    return similar_products


In [20]:
def find_manhattan_distance(sim_matrix, index, n=10):   
     
    # Getting Score and Index
    result = list(enumerate(sim_matrix[index]))

    # Sorting the Score and taking top 10 products
    sorted_result = sorted(result,key=lambda x:x[1],reverse=False)[1:10+1]
    
    # Mapping index with data
    similar_products =  [{'value': unique_df.iloc[x[0]]['Product Name'], 'score' : round(x[1], 2)} for x in sorted_result]
    
    return similar_products

In [24]:
def get_recommendation_cv(product_id, df, similarity, n=10):

    row = df.loc[df['Product Name'] == product_id]
    index = list(row.index)[0]
    description = row['desc_lower'].loc[index]

    #Create vector using Count Vectorizer

    count_vector = count_vectorizer.fit_transform(desc_list)
    
    if similarity == "cosine":
        sim_matrix = cosine_similarity(count_vector)
        products = find_similarity(sim_matrix , index)
        
    elif similarity == "manhattan":
        sim_matrix = manhattan_distances(count_vector)
        products = find_manhattan_distance(sim_matrix , index)
        
    else:
        sim_matrix = euclidean_distances(count_vector)
        products = find_euclidean_distances(sim_matrix , index)

    return products

In [27]:
# Cosine Similarity 
product_id = 'Vickerman 14" Finial Drop Christmas Ornaments, Pack of 2'
get_recommendation_cv(product_id, unique_df, similarity = "cosine", n=10)


[{'value': 'Fancyleo Christmas Glasses Frames 2 Pack Glittered Eyeglasses Glasses Set No Lens Kids Family Xmas Party Ornaments Gift',
  'score': 0.28},
 {'value': 'storefront christmas LED Decoration Light Gold Color Star Shape Vine Wedding Party event',
  'score': 0.2},
 {'value': '16 inches 40 inches "MERRY CHRISTMAS" Letter Foil Inflated Balloon Float Helium Aluminum Balloons for Christmas Decoration',
  'score': 0.19},
 {'value': '12ct Light Gunmetal Gray Shatterproof 4-Finish Christmas Ball Ornaments 4" (100mm)',
  'score': 0.19},
 {'value': 'PeanutsÃƒâ€šÃ‚ Valentine Sign Craft Kit (Pack of 12)',
  'score': 0.13},
 {'value': 'Simplicity 3 Pack Elastic Hair Ties White/Pink/Pink Leopard, 24 Count',
  'score': 0.13},
 {'value': '3 Pack Newbee Fashion- "Butterfly" Thin Design Gold Emblem Reading Glasses with Lanyard +1.75',
  'score': 0.12},
 {'value': 'Paper Mate Write Bros. Grip Mechanical Pencil, 0.7mm 5 ea (Pack of 2)',
  'score': 0.11},
 {'value': 'Christopher Radko Glass Plum Fr

In [28]:
# Manhattan Similarity 
get_recommendation_cv(product_id, unique_df, similarity = "manhattan", n=10)

[{'value': 'Stepping Stones', 'score': 43.0},
 {'value': 'Global Portuguese', 'score': 43.0},
 {'value': 'Polo Blue by Ralph Lauren', 'score': 43.0},
 {'value': 'Auburn Leathercrafters Tuscany Leather Dog Collar', 'score': 45.0},
 {'value': 'Leftover Salmon', 'score': 45.0},
 {'value': 'Good (Vinyl)', 'score': 45.0},
 {'value': 'Drunken Monkeys', 'score': 45.0},
 {'value': 'DuraTech Roof Support Trim', 'score': 47.0},
 {'value': 'Amerlite Niche Sealing Ring', 'score': 47.0},
 {'value': 'Learning and Performance in Corrections', 'score': 47.0}]

In [29]:
# Euclidean Similarity 
get_recommendation_cv(product_id, unique_df, similarity = "euclidean", n=10)

[{'value': 'Polo Blue by Ralph Lauren', 'score': 9.0},
 {'value': 'Auburn Leathercrafters Tuscany Leather Dog Collar', 'score': 9.11},
 {'value': 'Global Portuguese', 'score': 9.11},
 {'value': 'Stepping Stones', 'score': 9.22},
 {'value': 'Always in My Heart', 'score': 9.22},
 {'value': 'Leftover Salmon', 'score': 9.22},
 {'value': 'Good (Vinyl)', 'score': 9.22},
 {'value': 'Drunken Monkeys', 'score': 9.22},
 {'value': 'Learning and Performance in Corrections', 'score': 9.43},
 {'value': 'Chasing Hamburg (Vinyl)', 'score': 9.43}]

In [34]:
# Comparing similarity to get the top matches using TF-IDF

def get_recommendation_tfidf(product_id, df, similarity, n=10):

    row = df.loc[df['Product Name'] == product_id]
    index = list(row.index)[0]
    description = row['desc_lower'].loc[index]

    #Create vector using tfidf
    
    tfidf_matrix = tfidf.fit_transform(desc_list)
    
    if similarity == "cosine":
        sim_matrix = cosine_similarity(tfidf_matrix)
        products = find_similarity(sim_matrix , index)
        
    elif similarity == "manhattan":
        sim_matrix = manhattan_distances(tfidf_matrix)
        products = find_manhattan_distance(sim_matrix , index)
        
    else:
        sim_matrix = euclidean_distances(tfidf_matrix)
        products = find_euclidean_distances(sim_matrix , index)

    return products

In [35]:
# Cosine Similarity 
get_recommendation_tfidf(product_id, unique_df, similarity = "cosine", n=10)


[{'value': 'Fancyleo Christmas Glasses Frames 2 Pack Glittered Eyeglasses Glasses Set No Lens Kids Family Xmas Party Ornaments Gift',
  'score': 0.07},
 {'value': 'storefront christmas LED Decoration Light Gold Color Star Shape Vine Wedding Party event',
  'score': 0.05},
 {'value': '12ct Light Gunmetal Gray Shatterproof 4-Finish Christmas Ball Ornaments 4" (100mm)',
  'score': 0.05},
 {'value': '16 inches 40 inches "MERRY CHRISTMAS" Letter Foil Inflated Balloon Float Helium Aluminum Balloons for Christmas Decoration',
  'score': 0.05},
 {'value': 'Is It To Late To Be Good Grinch Christmas Mens Tank Top Shirt',
  'score': 0.02},
 {'value': 'Christopher Radko Glass Plum Frosty Snowman Christmas Ornament #1017624',
  'score': 0.02},
 {'value': 'CMFUN Watercolor Brush Creative Flower Made with Ink Hand Painting for Your Designs Pillowcase 20x20 inch',
  'score': 0.02},
 {'value': 'SKIN DECAL FOR OtterBox Symmetry Samsung Galaxy S7 Case - Christmas Snowflake Blue Ornaments DECAL, NOT A CAS

In [36]:
# Manhattan Similarity 
get_recommendation_tfidf(product_id, unique_df, similarity = "manhattan", n=10)


[{'value': 'Stepping Stones', 'score': 11.05},
 {'value': 'Global Portuguese', 'score': 11.42},
 {'value': 'Polo Blue by Ralph Lauren', 'score': 11.68},
 {'value': 'DuraTech Roof Support Trim', 'score': 12.06},
 {'value': 'Silver Metallic Foil Streamers', 'score': 12.07},
 {'value': 'Pro Pinstripe Jersey Sleeveles', 'score': 12.07},
 {'value': 'Amerlite Niche Sealing Ring', 'score': 12.07},
 {'value': 'Leftover Salmon', 'score': 12.3},
 {'value': 'Drunken Monkeys', 'score': 12.32},
 {'value': "Alfred's Drum Method, Book 1", 'score': 12.41}]

In [37]:
# Euclidean Similarity 
get_recommendation_tfidf(product_id, unique_df, similarity = "euclidean", n=10)


[{'value': 'Fancyleo Christmas Glasses Frames 2 Pack Glittered Eyeglasses Glasses Set No Lens Kids Family Xmas Party Ornaments Gift',
  'score': 1.37},
 {'value': 'storefront christmas LED Decoration Light Gold Color Star Shape Vine Wedding Party event',
  'score': 1.38},
 {'value': '12ct Light Gunmetal Gray Shatterproof 4-Finish Christmas Ball Ornaments 4" (100mm)',
  'score': 1.38},
 {'value': '16 inches 40 inches "MERRY CHRISTMAS" Letter Foil Inflated Balloon Float Helium Aluminum Balloons for Christmas Decoration',
  'score': 1.38},
 {'value': 'Is It To Late To Be Good Grinch Christmas Mens Tank Top Shirt',
  'score': 1.4},
 {'value': 'Christopher Radko Glass Plum Frosty Snowman Christmas Ornament #1017624',
  'score': 1.4},
 {'value': 'CMFUN Watercolor Brush Creative Flower Made with Ink Hand Painting for Your Designs Pillowcase 20x20 inch',
  'score': 1.4},
 {'value': 'SKIN DECAL FOR OtterBox Symmetry Samsung Galaxy S7 Case - Christmas Snowflake Blue Ornaments DECAL, NOT A CASE',

In [40]:
def get_recommendation_word2vec(product_id, df, similarity, n=10):
    
    row = df.loc[df['Product Name'] == product_id]
    input_index = list(row.index)[0]
    description = row['desc_lower'].loc[input_index]

    #create vectors for each desc using word2vec
    vector_matrix = np.empty((len(desc_list), 300))
    for index, each_sentence in enumerate(desc_list):
        sentence_vector = np.zeros((300,))
        count  = 0
        for each_word in each_sentence.split():
            try:
                sentence_vector += word2vecModel[each_word]
                count += 1
            except:
                continue
        
        vector_matrix[index] = sentence_vector
        
    
    if similarity == "cosine":
        sim_matrix = cosine_similarity(vector_matrix)
        products = find_similarity(sim_matrix , input_index)
        
    elif similarity == "manhattan":
        sim_matrix = manhattan_distances(vector_matrix)
        products = find_manhattan_distance(sim_matrix , input_index)
        
    else:
        sim_matrix = euclidean_distances(vector_matrix)
        products = find_euclidean_distances(sim_matrix , input_index)

    return products

In [41]:
# Cosine Similarity 
get_recommendation_word2vec(product_id, unique_df, similarity = "cosine", n=10)


[{'value': 'Fancyleo Christmas Glasses Frames 2 Pack Glittered Eyeglasses Glasses Set No Lens Kids Family Xmas Party Ornaments Gift',
  'score': 0.85},
 {'value': 'Red Gift Wrap Bag by K-Kraft (Festive Red Stripes, 13 x 17.5 inches)',
  'score': 0.85},
 {'value': '12 Pcs 3D Butterfly Stickers DIY Mural Art Decal Wall Stickers Crafts Wall Paper Decor (Yellow Series)',
  'score': 0.84},
 {'value': 'Cat Mod Garden Complex - Unfinished/Black', 'score': 0.83},
 {'value': 'Three Things That Makes Good Coffee: Sugar, Sugar and More Sugar Wall Decal - wall decal, sticker, quotes and sayings - W5082 - Light pink, 35in x 18in',
  'score': 0.82},
 {'value': 'Halloween Bats Wall Decal - Wall Sticker, Vinyl Wall Art, Home Decor, Wall Mural - W5014 - Royal blue, 16in x 10in',
  'score': 0.82},
 {'value': 'Hibiscus Flowers Wall Decal - Wall Sticker, Vinyl Wall Art, Home Decor, Wall Mural - 2186 - 8in x 19in, Dark green',
  'score': 0.82},
 {'value': 'Abstract Heart Plant Wall Decal - Wall Sticker, Vi

In [42]:
# Manhattan Similarity 
get_recommendation_word2vec(product_id, unique_df, similarity = "manhattan", n=10)


[{'value': 'storefront christmas LED Decoration Light Gold Color Star Shape Vine Wedding Party event',
  'score': 458.13},
 {'value': '8 1/2 x 14 Cardstock - Crystal Metallic (500 Qty.)',
  'score': 488.19},
 {'value': 'Cavalier Spaniel St. Patricks Day Shamrock Mouse Pad&#44; Hot Pad Or Trivet',
  'score': 497.0},
 {'value': "Call of the Wild Howling the Full Moon Women's Racerback Alpha Wolf",
  'score': 509.22},
 {'value': 'Fringe Table Skirt Purple 9 ft x 29 inches Pkg/1',
  'score': 516.08},
 {'value': 'Trend Enterprises T-83315 1.25 in. Holiday Pals & Peppermint Scratch N Sniff Stinky Stickers&#44; Large Round',
  'score': 522.0},
 {'value': "Allwitty 1039 - Women's T-Shirt Ipac Pistol Gun Apple Iphone Parody",
  'score': 525.03},
 {'value': 'Clear 18 Note Acrylic Box Musical Paperweight - Light My Fire',
  'score': 526.08},
 {'value': 'Handcrafted Ercolano Music Box Featuring "Luncheon of the Boating Party" by Renoir, Pierre Auguste - New YorkNew York',
  'score': 527.88},
 {'va

In [43]:
# Euclidean Similarity 
get_recommendation_word2vec(product_id, unique_df, similarity = "euclidean", n=10)


[{'value': 'storefront christmas LED Decoration Light Gold Color Star Shape Vine Wedding Party event',
  'score': 32.76},
 {'value': '8 1/2 x 14 Cardstock - Crystal Metallic (500 Qty.)',
  'score': 34.82},
 {'value': 'Cavalier Spaniel St. Patricks Day Shamrock Mouse Pad&#44; Hot Pad Or Trivet',
  'score': 35.78},
 {'value': "Call of the Wild Howling the Full Moon Women's Racerback Alpha Wolf",
  'score': 36.09},
 {'value': 'Fringe Table Skirt Purple 9 ft x 29 inches Pkg/1', 'score': 36.28},
 {'value': "New Way 075 - Men's Sleeveless Fbi Female Body Inspector",
  'score': 37.43},
 {'value': 'Trend Enterprises T-83315 1.25 in. Holiday Pals & Peppermint Scratch N Sniff Stinky Stickers&#44; Large Round',
  'score': 37.51},
 {'value': 'Platinum 5 mm Comfort Fit Half Round Wedding Band - Size 9.5',
  'score': 37.6},
 {'value': 'Christmas Presents Between Friends Chinese Crested Black Sticky Note Holder',
  'score': 37.78},
 {'value': 'Tulane Green Wave Muscle Mug', 'score': 37.85}]

In [44]:
# Comparing similarity to get the top matches using Glove pretrained model

def get_recommendation_glove(product_id, df, similarity, n=10):

    row = df.loc[df['Product Name'] == product_id]
    input_index = list(row.index)[0]
    description = row['desc_lower'].loc[input_index]

    #using glove embeddings to create vectors
    vector_matrix = np.empty((len(desc_list), 300))
    for index, each_sentence in enumerate(desc_list):
        sentence_vector = np.zeros((300,))
        count  = 0
        for each_word in each_sentence.split():
            try:
                sentence_vector += glove_model[each_word]
                count += 1
                
            except:
                continue
        
        vector_matrix[index] = sentence_vector
    
    
    if similarity == "cosine":
        sim_matrix = cosine_similarity(vector_matrix)
        products = find_similarity(sim_matrix , input_index)
        
    elif similarity == "manhattan":
        sim_matrix = manhattan_distances(vector_matrix)
        products = find_manhattan_distance(sim_matrix , input_index)
        
    else:
        sim_matrix = euclidean_distances(vector_matrix)
        products = find_euclidean_distances(sim_matrix , input_index)

    return products


In [45]:
# Cosine Similarity 
get_recommendation_glove(product_id, unique_df, similarity = "cosine", n=10)


[{'value': 'Red Gift Wrap Bag by K-Kraft (Festive Red Stripes, 13 x 17.5 inches)',
  'score': 0.94},
 {'value': 'Cat Mod Garden Complex - Unfinished/Black', 'score': 0.93},
 {'value': 'CafePress - FIN Nuts Squirrel - Cute Infant Baby Football Bodysuit',
  'score': 0.93},
 {'value': 'American Foxhound Christmas Sticky Note Holder BB8433SN',
  'score': 0.93},
 {'value': 'CHAIM Street Sign Childrens Name Room Sign | Indoor/Outdoor | Â\xa024" Wide',
  'score': 0.93},
 {'value': 'All Weather Cornhole Bags - Set of 8', 'score': 0.93},
 {'value': '12 Pcs 3D Butterfly Stickers DIY Mural Art Decal Wall Stickers Crafts Wall Paper Decor (Yellow Series)',
  'score': 0.93},
 {'value': 'MightySkins Skin for Fitbit Charge 3 - Cartoon Mania | Protective, Durable, and Unique Vinyl Decal wrap cover | Easy To Apply, Remove, and Change Styles | Made in the USA',
  'score': 0.93},
 {'value': 'Ecclesiastes 8:15 so I commend the enjoymentÃ¢â‚¬Â¦ Vinyl Decal Sticker Quote - Small - Metallic Gold',
  'score': 

In [47]:
# Manhattan Similarity 
get_recommendation_glove(product_id, unique_df, similarity = "manhattan", n=10)


[{'value': 'Is It To Late To Be Good Grinch Christmas Mens Tank Top Shirt',
  'score': 1187.12},
 {'value': 'Platinum 5 mm Comfort Fit Half Round Wedding Band - Size 9.5',
  'score': 1191.95},
 {'value': "New Way 075 - Men's Sleeveless Fbi Female Body Inspector",
  'score': 1202.32},
 {'value': 'storefront christmas LED Decoration Light Gold Color Star Shape Vine Wedding Party event',
  'score': 1211.6},
 {'value': '8 1/2 x 14 Cardstock - Crystal Metallic (500 Qty.)',
  'score': 1232.94},
 {'value': "Allwitty 1039 - Women's T-Shirt Ipac Pistol Gun Apple Iphone Parody",
  'score': 1234.72},
 {'value': "Call of the Wild Howling the Full Moon Women's Racerback Alpha Wolf",
  'score': 1240.1},
 {'value': '031 - Unisex Long-Sleeve T-Shirt Disobey V For Vendetta Anonymous Fawkes Mask',
  'score': 1252.5},
 {'value': 'Skin Decal For Beats By Dr. Dre Beats Pill Plus / Sugar Skull Girl',
  'score': 1256.38},
 {'value': "1042 - Unisex Long-Sleeve T-Shirt Free Hugs And Maybe A Kiss If You're Luck